In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import joblib
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta

In [2]:
#moving average
def mac(ticker,sd,ed):
    df=yf.download(ticker,sd,ed)
    df['LTMA_today_end']=talib.SMA(df['Close'],timeperiod=50)
    df['STMA_today_end']=talib.SMA(df['Close'],timeperiod=20)
    df['tomorrow_position']=np.nan
    le=df['STMA_today_end']>df['LTMA_today_end']
    se=df['STMA_today_end']<df['LTMA_today_end']
    df.loc[le,'tomorrow_position']=1
    df.loc[se,'tomorrow_position']=-1
    df['tomorrow_return']=((df['Close'].shift(-1)-df['Open'].shift(-1))/df['Open'].shift(-1))*df['tomorrow_position']
    df=df.dropna()
    return df

In [3]:
#volsurge
def volsurge(ticker,sd,ed):
    df=yf.download(ticker,sd,ed)
    df['vol_avg']=df['Volume'].rolling(20).mean()
    df['vol_avg']=df['vol_avg'].shift(1)
    df['vol_std']=df['Volume'].rolling(20).std()
    df['vol_std']=df['vol_std'].shift(1)
    df['open_avg']=df['Open'].rolling(20).mean()
    df['open_avg']=df['open_avg'].shift(1)
    df['open_std']=df['Open'].rolling(20).std()
    df['open_std']=df['open_std'].shift(1)
    df['tomorrow_position']=0
    le=(df['Volume']>=df['vol_avg']+df['vol_std'])&(df['Open']>=df['open_avg']+df['open_std'])
    se=(df['Volume']<=df['vol_avg']-df['vol_std'])&(df['Open']<=df['open_avg']-df['open_std'])
    df.loc[le,'tomorrow_position']=1
    df.loc[se,'tomorrow_position']=-1
    df['tomorrow_return']=((df['Close'].shift(-1)-df['Open'].shift(-1))/df['Open'].shift(-1))*df['tomorrow_position']
    df=df.dropna()
    return df


In [4]:
#rangetrading
def range(ticker,sd,ed):
    df=yf.download(ticker,sd,ed)
    df['high_avg']=np.nan
    df['low_avg']=np.nan
    df['high_avg']=df['High'].rolling(20).max()
    df['high_avg']=df['high_avg'].shift(1)
    df['low_avg']=df['Low'].rolling(20).min()
    df['low_avg']=df['low_avg'].shift(1)
    df['tom_position']=0
    le=df['Close']>df['high_avg']
    se=df['Close']<df['low_avg']
    df.loc[le,'tom_position']=1
    df.loc[se,'tom_position']=-1
    df['tomorrow_return']=df['tom_position']*((df['Close'].shift(1)-df['Open'].shift(1))/df['Open'].shift(1))
    df=df.dropna()
    df.drop(columns=['high_avg','low_avg'],inplace=True)
    return df



In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

    

In [6]:
def forest(df):
    x=df.iloc[:,0:-1]#features
    y=df.iloc[:,-1]#target(tomorrow's return)
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
    pipe=Pipeline([
        ('scaler',StandardScaler()),
        ('forest',RandomForestRegressor(n_jobs=-1))
    ])
    hyperparam={'forest__n_estimators':[1000,2000,5000]}
    gs=GridSearchCV(cv=5,estimator=pipe,param_grid=hyperparam,scoring='neg_mean_squared_error')
    gs.fit(x_train,y_train)
    best=gs.best_estimator_
    best.fit(x_train,y_train)
    return best
    

In [1]:
def mainfunc():
    p1=0
    p2=0
    p3=0
    ticker=input("Enter ticker of the company you want to trade")
    #getting start and end date
    enddate=input("Enter present date in yy-mm-dd format")
    end_date_obj = datetime.strptime(enddate, '%Y-%m-%d')
    start_date_obj = end_date_obj - relativedelta(years=1) #date one year from the date entered by user
    startdate = start_date_obj.strftime('%Y-%m-%d')
    #preparing dataframes
    dfmac=mac(ticker,startdate,enddate)
    dfvolsurge=volsurge(ticker,startdate,enddate)
    dfrange=range(ticker,startdate,enddate)
    #training and saving models
    m1=forest(dfmac)
    m2=forest(dfvolsurge)
    m3=forest(dfrange)
    joblib.dump(m1,'macpredictor')
    joblib.dump(m2,'volsurgepredictor')
    joblib.dump(m3,'rangepredictor')
    #getting data from user
    o=float(input("enter today's open"))
    h=float(input("enter today's high"))
    l=float(input("enter today's low"))
    c=float(input("enter today's close"))
    ac=float(input("enter today's adjusted close"))
    v=float(input("enter today's volume"))
    ltma=float(input("Enter 50 day long term moving average as of today's end"))
    stma=float(input("Enter 20 day short term moving average as of today's end"))
    if ltma<stma:
        p1=1
    else:
        p1=-1
    avgvol=float(input("Enter average volume for past 20 days"))
    stdvol=float(input("Enter standard deviation of volume for past 20 days"))
    avgopen=float(input("Enter average of open for past 20 days"))
    stdopen=float(input("Enter standard deviation of open for past 20 days"))
    if v>avgvol+stdvol and o>avgopen+stdopen:
        p2=1
    else:
        p2=-1
    avghigh=float(input("Enter average of high for the past 20 days"))
    avglow=float(input("Enter average of low for the past 20 days"))
    if c>avghigh:
        p3=1
    elif c<avglow:
        p3=-1
    m1=joblib.load('macpredictor')
    m2=joblib.load('volsurgepredictor')
    m3=joblib.load('rangepredictor')
    macpred=m1.predict([[o,h,l,c,ac,v,ltma,stma,p1]])
    vspred=m2.predict([[o,h,l,c,ac,v,avgvol,stdvol,avgopen,stdopen,p2]])
    ranpred=m3.predict([[o,h,l,c,ac,v,p3]])
    if macpred[0]>vspred[0] and macpred[0]>ranpred[0]:
        print("Most Suitable Strategy:Moving Average Crossover")
        print("Predicted Return for buying tomorrow:",macpred[0])
    elif vspred[0]>macpred[0] and vspred[0]>ranpred[0]:
        print("Most Suitable Strategy:Volume Surge Trading")
        print("Predicted Return for buying tomorrow:",vspred[0])
    else:
        print("Most Suitable Strategy:Range Trading")
        print("Predicted Return for buying tomorrow:",ranpred[0])
        
        

In [11]:
print("There are three strategies- Moving Average Crossover, Volume Surge, and Range Trading strategy")
print("For the ticker you provide, the model will train to predict tomorrow's return, using today's data, using each of these 3 strategies")
print("The strategy that will give the highest return for day trading tomorrow will be presented after calculation")
print("For the model to work, you need to provide some of the data at today's close")
func()

There are three strategies- Moving Average Crossover, Volume Surge, and Range Trading strategy
For the ticker you provide, the model will train to predict tomorrow's return, using today's data, using each of these 3 strategies
The strategy that will give the highest return for day trading tomorrow will be presented after calculation
For the model to work, you need to provide some of the data at today's close


Enter ticker of the company you want to trade RELIANCE.NS
Enter present date in yy-mm-dd format 2020-5-12


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/0f/qwg2wd5d1rbdpdh684rl_c8h0000gn/T/ipykernel_64635/1541381835.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['high_avg','low_avg'],inplace=True)


enter today's open 2200.91
enter today's high 2290.9
enter today's low 2979.3
enter today's close 2215.4
enter today's adjusted close 2215.4
enter today's volume 2509088
Enter 50 day long term moving average as of today's end 2212.14
Enter 20 day short term moving average as of today's end 2213.31
Enter average volume for past 20 days 2409899
Enter standard deviation of volume for past 20 days 2.3
Enter average of open for past 20 days 2215.89
Enter standard deviation of open for past 20 days 3.1
Enter average of high for the past 20 days 2288.12
Enter average of low for the past 20 days 2970.1


Most Suitable Strategy:Range Trading
Predicted Return for buying tomorrow: 0.010835346649989386


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
